In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pretty_midi
from mido import Message, MidiFile, MidiTrack
import pandas as pd
import torch as torch
from tqdm import tqdm

In [ ]:
my_data = pd.read_csv('maestro-v3.0.0/maestro-v3.0.0.csv')

In [ ]:
my_data.iloc[45]

In [ ]:
target_size = 400
path_beginning = "maestro-v3.0.0/"
train = []
test = []
validation = []
for i in tqdm(range(0, 1276)):
  path_part = my_data.iloc[i]["midi_filename"]
  split = my_data.iloc[i]["split"]
  path = path_beginning + path_part
  midi_data = pretty_midi.PrettyMIDI(path)
  tempo = midi_data.estimate_tempo()
  roll = midi_data.get_piano_roll(tempo/60).T
  over = False
  r = np.zeros(128)
  while over == False:
    m = roll[0]
    if np.array_equal(m,r):
      roll = roll[1:]
    else:
      over = True
  roll = np.where(roll > 0, 1, 0)
  sev_roll = []
  if roll.shape[0]<target_size:
    while roll.shape[0]<target_size:
      roll = np.vstack((roll,roll))
    roll = roll[:target_size]
    sev_roll.append(np.array([roll,tempo], dtype="object"))
  else:
    num_arrays = roll.shape[0]//target_size
    leftover = roll.shape[0]%target_size
    for l in range(0, num_arrays):
      start = target_size*l
      end = target_size*(l+1)
      sev_roll.append(np.array([roll[start:end],tempo], dtype="object"))
    if leftover >=200:
      leftover_roll = roll[end:]
      while leftover_roll.shape[0]<target_size:
        leftover_roll = np.vstack((leftover_roll,leftover_roll))
      leftover_roll = leftover_roll[:target_size]
      sev_roll.append(np.array([leftover_roll,tempo], dtype="object"))
  for roll in sev_roll:
    roll = np.array([torch.tensor(roll[0]), roll[1]], dtype="object")
    if split=="test":
      test.append(roll)
    elif split=="train":
      train.append(roll)
    else:
      validation.append(roll)

In [ ]:
torch.save(validation, "validation_tempo.csv" )
torch.save(test, "test_tempo.csv" )
torch.save(train, "train_tempo.csv" )